# HeatMapwithTime

## Generating weekly heatmap from 2023

In [6]:
import geopandas as gpd
import folium
from folium.plugins import HeatMapWithTime
import pandas as pd

# Preprocessing
crime_data = gpd.read_file('../../dataset/ALL 2023 AND UNTIL JAN29 2024.geojson')
crime_data = crime_data[crime_data['geometry'].notna()] # Remove rows with missing geometry
crime_data = crime_data[crime_data['CREATE_TIME_INCIDENT'].dt.year == 2023] # Remove data not from 2023
crime_data.crs = 'EPSG:3857' # CRS represents the coordinate system. crime_data initally is set to have CRS of EPSG:4326 but this is actually wrong. Here I set the CRS to be the correct one.
crime_data = crime_data.to_crs(epsg=4326) # change the coordinate system to EPSG:4326 or (longitude, latitude). Geocoder only accepts (long, lat).
unwanted_locations = ['LEON CO DETENTION FACILITY', 'TALLAHASSEE POLICE DEPARTMENT', 'LCSO', 'TPD', 'LEON CO SHERIFFS OFFICE', 'IMMIGRATION & CUSTOMS ENFORCEM']
crime_data = crime_data[~crime_data['LOCATION_TEXT'].isin(unwanted_locations)]
unwanted_categories = ["COMMUNITY POLICING", "ADMIN", "RECOVERED PROP", "AUTO THEFT RECOVERY","UNABLE TO VERIFY", "TRAFFIC"]
crime_data = crime_data[~crime_data['DISPO_TEXT'].isin(unwanted_categories)]
# Convert 'CREATE_TIME_INCIDENT' to datetime if not already done
crime_data['CREATE_TIME_INCIDENT'] = pd.to_datetime(crime_data['CREATE_TIME_INCIDENT'])

# Example conversion from GeoDataFrame to DataFrame with lat/lon columns
crime_data['latitude'] = crime_data.geometry.y
crime_data['longitude'] = crime_data.geometry.x
crime_data['date'] = crime_data['CREATE_TIME_INCIDENT'].dt.date


# Aggregate data by week
# Note: Adjusting from .dt.date to .dt.isocalendar().week for weekly data
weekly_data = crime_data.groupby(crime_data['CREATE_TIME_INCIDENT'].dt.isocalendar().week)

# Initialize a map:
m = folium.Map(location=[crime_data['latitude'].mean(), crime_data['longitude'].mean()], zoom_start=12)

# Prepare data for the HeatMapWithTime plugin
data = [
    [[row['latitude'], row['longitude']] for index, row in group.iterrows()] 
    for week, group in weekly_data
]

# Generate index for time slider
index = [f"Week {week}" for week, group in weekly_data]

# Create a HeatMapWithTime
HeatMapWithTime(data, index=index, auto_play=True, max_opacity=0.8).add_to(m)

# Save or display the map
m.save('Crime_HeatMapWithTime_Weekly.html')
# Or display in a Jupyter notebook environment using: display(m)

# Comments on the heatmap
We note that the large scale shape is somewhat stable, which suggest that there are places where there are reports constantly.

# A few places where the crimes aggregate
- N. Monroe Street near I-10 (The south of N. Monroe seems to get constant reports in general, but the part near I-20 seems especially bad)
-
- W. Pensacola Street around Westend Square. Not only Westland Square itself, the area around it seems to get constant reports.
- Governer's Mall and S. Magnolia Drive next to it.
- W. Tenessee Street between Stadium Dr. and Ocala Rd.
- Blountstown Street and the surrounding area
- In larger scale, the area aroudn the campus seems to have more reports in general (although I might be biased as this is common thing being said among FSU students).

This is not an exhaustive list. Perhaps unsurprising, but we can see some of the places above are big shopping malls and major streets.
